In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.optimize

import trfp

In [ ]:
tr_x = np.array([0] + [1.75 * np.sin(2*np.pi/4*i) for i in np.arange(4)] + [3.5 * np.sin(2*np.pi/12*i) for i in np.arange(12)])
tr_y = np.array([0] + [-1.75 * np.cos(2*np.pi/4*i) for i in np.arange(4)] + [-3.5 * np.cos(2*np.pi/12*i) for i in np.arange(12)])

fp6_x = np.array([-3, 0, 3, -3, 0, 3])
fp6_x_2 = np.array([-4, -1, 2, -4, -1, 2])
fp6_y = np.array([7.7, 7.7, 7.7, -7.7, -7.7, -7.7])

fp4_x = np.array([0, 3, 0, 3])
fp4_y = np.array([7.7, 7.7, -7.7, -7.7])

def multipole(N,s,A,x,y):
    r = np.sqrt(x**2+y**2)
    theta = np.arctan2(y,x)
    if(s==0):
        By = 61.79e6 + A * (r/4.5)**N * np.cos(N*theta)
        Bx = A * (r/4.5)**N * np.sin(N*theta)
    if(s==1):
        By = 61.79e6 + A * (r/4.5)**N * np.sin(N*theta)
        Bx = -A * (r/4.5)**N * np.cos(N*theta)
    B = np.sqrt(Bx**2 + By**2) - 61.79e6
    return B

def lin_fit(x, a, b):
    return a + b*x

Ns = [0,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8];
ss = [0,0,1,1,0,0,1,0,1,0,1,0,1,0,1,0,1];

THETA_TR = np.linalg.pinv(np.transpose(np.array([multipole(Ns[i],ss[i],1,tr_x,tr_y) for i in np.arange(17)])))

THETA_FP_6 = np.array([np.array([1, 1, 1, 1, 1, 1])/6.,  # dipole
                       np.array([1, 0, -1, 1, 0, -1])/-12.*4.5,  # n quad
                       np.array([1, 1, 1, -1, -1, -1])/46.2*4.5,  # s quad
                       np.array([1, 0, -1, -1, 0, 1])/-92.4*4.5**2,  # s sext
                       np.array([1, -2, 1, 1, -2, 1])/18.*4.5**2,  # n sext
                       np.array([1, -2, 1, -1, 2, -1])/-138.6*4.5**3]  # NOT oct
                     )

THETA_FP_4 = np.array([np.array([1, 0, 1, 0])/2.,  # dipole
                       np.array([1, -1, 1, -1])/-6.*4.5,  # n quad
                       np.array([1, 1, -1, -1])/30.8*4.5,  # s quad
                       np.array([1, -1, -1, 1])/-46.2*4.5**2]  # sext
                     )

def jacobian_calc(probes, offset):
    if probes == 6:
        THETA_FP = THETA_FP_6
        if offset:
            fp_x = fp6_x_2
            fp_y = fp6_y
        else:
            fp_x = fp6_x
            fp_y = fp6_y
    else:
        probes = 4
        THETA_FP = THETA_FP_4
        fp_x = fp4_x
        fp_y = fp4_y
        
    As = np.arange(-10,10)

    dfp_dtr = np.zeros((probes, probes))
    
    for ii in np.arange(probes):
        N = Ns[ii]
        s = ss[ii]

        tr_out = np.empty((len(As),probes))
        tr_out[:] = np.nan
        fp_out = np.empty((len(As),probes))
        fp_out[:] = np.nan

        for jj in np.arange(len(As)):
            A = As[jj]
            B_tr = multipole(N, s, A, tr_x, tr_y)
            B_fp = multipole(N, s, A, fp_x, fp_y)

            tr_out[jj,:] = np.matmul(THETA_TR, B_tr)[0:probes]
            fp_out[jj,:] = np.matmul(THETA_FP, B_fp)

        for kk in np.arange(probes):
            coeffs, covar = scipy.optimize.curve_fit(lin_fit, As, fp_out[:,kk])
            dfp_dtr[kk,ii] = coeffs[1]
    return dfp_dtr[0:5,0:5]

In [ ]:
# look at fp vs tr (tr is good proxy for actual at low m)

probes = 6
offset = False

dfp_dtr = jacobian_calc(probes, offset)

print np.round(dfp_dtr, 3)
print np.round(np.linalg.inv(dfp_dtr), 3)
# print np.round(np.linalg.pinv(dfp_dtr[0:5,0:5]), 3)
# print np.round(np.matmul(np.linalg.pinv(dfp_dtr[0:5,0:5]),
#                          dfp_dtr[0:5,0:5]
#                         ),
#                3)
        


In [ ]:
# test Jacobian

probes = 6
offset = True

J_6_probe = trfp.J_6_PROBE
J_6_probe_offset = trfp.J_6_PROBE_OFFSET
J_4_probe = trfp.J_4_PROBE

# print np.round(dfp_dtr, 3)
print np.round(J_4_probe, 3)

B_tr = multipole(1, 1, 4.7, tr_x, tr_y) + multipole(2, 1, -1.35, tr_x, tr_y)
B_fp = multipole(1, 1, 4.7, fp4_x, fp4_y) + multipole(2, 1, -1.35, fp4_x, fp4_y)

m_tr = np.matmul(THETA_TR, B_tr)
m_fp = np.matmul(THETA_FP_4, B_fp)

print np.round(m_tr[0:4], 3)
print np.round(m_fp, 3)

print np.round(np.matmul(J_4_probe, m_fp), 3)